In [30]:
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader

from energy_test import df_to_input_output

df = pd.read_pickle('data.pkl')
X, y = df_to_input_output(df)

N = 100
X, y = X[:N], y[:N]

X_train, X_test, y_train, y_test = map(torch.from_numpy, train_test_split(X, y, test_size=0.5, shuffle=False))
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([50, 7]),
 torch.Size([50, 7]),
 torch.Size([50, 1]),
 torch.Size([50, 1]))

In [32]:
device = "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_size, 512),
            nn.ReLU(),
            nn.Linear(512, out_size)
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

in_size, out_size = X.shape[-1], y.shape[-1]
model = NeuralNetwork(in_size, out_size).to(device)
model.double()

batch_size = 64

"""
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
"""

print(model)

Using cpu device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [33]:
def train(X_train, y_train, model, loss_fn, optimizer):
    size = y_train.shape[-1]
    model.train()
    for batch, (X, y) in enumerate(zip(X_train, y_train)):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(X_test, y_test, model, loss_fn):
    size = y_train.shape[-1]
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in zip(X_test, y_test):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    #test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(X_train, y_train, model, loss_fn, optimizer)
    test(X_test, y_test, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: -0.000000  [    7/    1]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)